### Constants and Imports

In [1]:
pickle_rel_path = '../datasets/notmnist/notMNIST.pickle'
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

### Import or generate data

In [2]:
file_dir = os.path.dirname(os.path.realpath('__file__'))
pickle_path = os.path.abspath(os.path.join(file_dir, pickle_rel_path))
print('Pickle location:',pickle_path)

Pickle location: C:\Users\lgess\Documents\repo\learning_ai\datasets\notmnist\notMNIST.pickle


In [3]:
with open(pickle_path,'rb') as file:
    datasets = pickle.load(file)
print(datasets.keys())

dict_keys(['valid_dataset', 'test_dataset', 'valid_labels', 'test_labels', 'train_labels', 'train_dataset'])


In [11]:
train_dataset = datasets['train_dataset']
orig_train_labels = datasets['train_labels'] 
validate_dataset = datasets['valid_dataset']
orig_validate_labels = datasets['valid_labels'] 
test_dataset = datasets['test_dataset']
orig_test_labels = datasets['test_labels'] 

### Setup data (convert to convenient representation)

In [5]:
def setup_labels(label_array, num_labels):
    return ((np.arange(num_labels) == label_array[:,None]).astype(np.float32))

num_labels = 10
train_labels = setup_labels(orig_train_labels, num_labels)
validate_labels = setup_labels(orig_validate_labels, num_labels)
test_labels = setup_labels(orig_test_labels, num_labels)

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

### Normalize data

Pre-normalized

### Define model

In [16]:
ops.reset_default_graph()

learning_rate = 0.05
batch_size = 5000

samples_shape = np.append(None,train_dataset.shape[1:])
targets_shape = np.append(None,train_labels.shape[1:])
weights_shape = np.append(num_labels,samples_shape[1:])
biasses_shape = [num_labels]

samples = tf.placeholder(tf.float32, shape=samples_shape)
targets = tf.placeholder(tf.float32, shape=targets_shape)
weights = tf.get_variable("W", shape=weights_shape, dtype=tf.float32)
biasses = tf.get_variable("B", shape=biasses_shape, dtype=tf.float32)
print(samples_shape,weights_shape,biasses_shape)

def weighted_inputs():
    axis_weights = np.arange(len(weights_shape))[1:]
    axis_samples = np.arange(len(samples_shape))[1:]
    axes = (axis_samples,axis_weights)
    return tf.tensordot(samples,weights,axes)+biasses

def activation_function(weighted_inputs):
    exp = tf.exp(weighted_inputs)
    return (tf.div(exp,tf.add(exp,1)))

def model():
    return activation_function(weighted_inputs())

def loss():
    return tf.reduce_mean(tf.square(tf.subtract(targets,model())))

def training_step():
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    return optimizer.minimize(loss())

[None 28 28] [10 28 28] [10]


### Train model

In [17]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [18]:
# Training loop
loss_vec = []
for i in range(100):
    rand_index = np.random.choice(len(train_dataset), size=batch_size)
    feed_dict={samples: train_dataset[rand_index], targets: train_labels[rand_index]}
    sess.run(training_step(), feed_dict)
    temp_loss = sess.run(loss(), feed_dict)
    if i%10 == 0:
        print(temp_loss)

0.30389214
0.2718571
0.243364
0.2247313
0.2076934
0.1950117
0.18407273
0.17651777
0.16924886
0.16473803


In [32]:
def show_sample(sample_image, label):
    plt.figure()
    plt.imshow(sample_image)
    print('Label:', label)

#index=0
#show_sample(validate_dataset[index],validate_labels[index])
#sess.run(model(),feed_dict={samples: [validate_dataset[index]]})
#show_sample(sess.run(weights[4]),'')